Install the mysql-connector-python and pandas

### Introduction
This notebook perform an ETL process, it start with get data from Excel file then transform data then generate relevant dataframes. After that clean data will load into corresponding tables in MySQL database.

In [1]:
# This script is intended to be run in a Jupyter notebook or similar environment
# You can install the required packages using pip
# Uncomment the following line if you haven't installed the packages yet
# !pip install openpyxl pandas mysql-connector-python

In [2]:
# Import necessary libraries
import openpyxl
import mysql.connector as connector
import pandas as pd     
import numpy as np

In [3]:
# Set up the connection to the MySQL database
try:
    connection = connector.connect(
        host="localhost",
        user="root",
        password="thanhhoa",
        database="littlelemon"
    )
    cursor = connection.cursor()
except connector.Error as er:
    print("Error Code ", er.errno)
    print("Error Message ", er.msg)

### 1. Get data from Excel file source and perform initial cleaning

In [4]:
# Read the Excel file and load the Orders sheet into a DataFrame
df = pd.read_excel("Raw Data/LittleLemonDB.xlsx", sheet_name="Orders")

Perform some initial cleaning

In [ ]:
# Drop the first column
df.drop(columns=["Row Number"], inplace=True)

# Rename the columns to remove leading spaces
df.columns = df.columns.str.strip()

# Drop duplicate rows based on the "Order ID" column
df = df.drop_duplicates(subset = ["Order ID"])

# Remove non-numeric characters from the Order ID column and Customer ID column then change the type to int
df['Order ID'] = df['Order ID'].str.replace('-','').astype(int)
df['Customer ID'] = df['Customer ID'].str.replace('-','').astype(int)

In [5]:
# Viewing few first rows in dataset
print("The first 5 rows of the dataset:")
print(df.head(5))

# Check shape of dataset
print("The shape of the dataset:")
print(df.shape)


The first 5 rows of the dataset:
   Row Number     Order ID Order Date Delivery Date  Customer ID  \
0           1  54-366-6861 2020-06-15    2020-03-26  72-055-7985   
1           2  63-761-3686 2020-08-25    2020-07-17  65-353-0657   
2           3  65-351-6434 2021-08-17    2020-04-24  90-876-6799   
3           4  36-917-2834 2021-08-14    2020-04-13  73-873-4827   
4           5  86-114-9232 2020-12-20    2021-02-02  80-927-5246   

     Customer Name        City      Country Postal Code Country Code  ...  \
0     Laney Fadden    Daruoyan        China    993-0031           CN  ...   
1  Giacopo Bramich      Ongjin  North Korea      216282           KP  ...   
2        Lia Bonar  Quince Mil         Peru      663246           PE  ...   
3   Merrill Baudon      Susaki        Japan    987-0352           JP  ...   
4     Tasia Fautly      Tobruk        Libya      351 01           LY  ...   

   Sales  Quantity  Discount  Delivery Cost  Course Name Cuisine Name  \
0  187.5         2    

### 2. Extract relevant data for each table then load to MySQL database
It is noted that the database schema already created using MySQL Workbench, so we do not create new tables here.
The database schema already has five tables as below and only need to fill in the data for those tables:
- Customers
- Menu_Items
- Deliveries
- Orders
- Bookings

#### 2.1 Prepare data for customers table

In [6]:
# Extract the relevant columns for customers
customers = df[["Customer ID", "Customer Name", "City", "Country"]].drop_duplicates()

Create customer table and add data into table

In [ ]:

# Insert data into the customers table
for i, row in customers.iterrows():
    sql = "insert into customers (CustomerID, Name, City, Country) values (%s, %s, %s, %s)"
    cursor.execute(sql,tuple(row))

connection.commit()

#### 2.2 Prepare data for menu_items table

In [ ]:
# Create a MenuItems data frame
menu_items = df[["Course Name", "Starter Name", "Desert Name", "Drink", "Sides"]].drop_duplicates()

In [ ]:
# Add a new column "MenuItemID" to the menu_items DataFrame
menu_items.insert(0, "MenuItemID", range(1, len(menu_items) + 1))
menu_items

In [190]:

# Insert data into the menu_items table
for i, row in menu_items.iterrows():
    sql = "insert into menu_items (Menu_Items_ID,CourseName, StarterName, DesertName, Drink, Sides) values (%s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))

connection.commit()


In [191]:
# Merge df with menu_items to get MenuItemID to create foreign key relationship
df = df.merge(menu_items, how="left", on=["Course Name", "Starter Name", "Desert Name", "Drink", "Sides"])

In [ ]:
# Printout the first 10 rows of the merged DataFrame to check the foreign key relationship
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df.head(10).iloc[:, -4:])

       Desert Name              Drink         Sides  MenuItemID
0    Greek yoghurt  Athens White wine         Tapas           1
1        Ice cream     Corfu Red Wine  Potato salad           2
2       Cheesecake     Italian Coffee    Bruschetta           3
3         Affogato      Roma Red wine      Focaccia           4
4  Turkish yoghurt  Ankara White Wine     Meatballs           5
5          Baklava     Turkish Coffee         Fries           6
6    Greek yoghurt  Athens White wine         Tapas           1
7        Ice cream     Corfu Red Wine  Potato salad           2
8       Cheesecake     Italian Coffee    Bruschetta           3
9         Affogato      Roma Red wine      Focaccia           4


#### 2.3 Prepare data for Order table

In [194]:
# Create Orders data frame
Orders = df[["Order ID", "Order Date", "Customer ID", "Cost", "Sales", "Discount", "MenuItemID", "Quantity"]].drop_duplicates()

In [195]:
# Check data types of the Order Date column
print(type(Orders["Order Date"][0]))

# Convert the Order Date to a string in the format YYYY-MM-DD
Orders["Order Date"] = Orders["Order Date"].dt.strftime('%Y-%m-%d')

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [196]:

# Insert data into the orders table
for i, row in Orders.iterrows():
    sql = "insert into orders (OrderID, OrderDate, CustomerID, Cost, Sales, Discount, MenuItemID, Quantity) values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))

In [ ]:
# Make persistent changes to the database
connection.commit()

### 2.4 Prepare data for deliveries table

In [205]:
# Extract the relevant columns for Deliveries
deliveries = df[["Delivery Date", "Order ID"]].drop_duplicates()

In [206]:
# Convert the Order Date to a string in the format YYYY-MM-DD
deliveries["Delivery Date"] = deliveries["Delivery Date"].dt.strftime('%Y-%m-%d')

In [207]:
# Delivery status is not in the original data, so we will create a new column with 
# random value of "Not initiate", "Delivered", "In Progress", "Cancelled"
status = ["Not initiate", "Delivered", "In Progress", "Cancelled"]
deliveries["Delivery Status"] = np.random.choice(status, size=len(deliveries))
deliveries["Deliveries_ID"] = range(1, len(deliveries) + 1)

In [ ]:
# View the first 5 rows of the deliveries DataFrame
print("The first 5 rows of the deliveries DataFrame:")
deliveries.head(5)

,Delivery Date,Order ID,Delivery Status,Deliveries_ID
0,2020-03-26,543666861,In Progress,1
1,2020-07-17,637613686,Not initiate,2
2,2020-04-24,653516434,Not initiate,3
3,2020-04-13,369172834,Cancelled,4
4,2021-02-02,861149232,Not initiate,5


In [209]:
# Insert data into the deliveries table
for i, row in deliveries.iterrows():
    sql = "insert into deliveries (DeliveryDate, OrderID, DeliveryStatus, Deliveries_ID) values (%s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))


In [ ]:
# Update persistent changes to the database
connection.commit()

#### 2.5Prepare data for Booking table
Source table do not have information about booking system, so simulated data is used instead.

In [211]:
# Create Bookings data frame
bookings = df[["Customer ID"]][:20].drop_duplicates()

In [ ]:
# Add BookingID and TableNo columns, format BookingDate
bookings['BookingID'] = range(1, len(bookings) + 1)
bookings['BookingDate'] = pd.to_datetime('2023-10-01') + pd.to_timedelta(np.random.randint(0, 30, size=len(bookings)), unit='d')
bookings['TableNo'] = np.random.randint(1, 11, size=len(bookings))

In [213]:
# Rearrange the columns to match the database schema
bookings = bookings.reindex(columns=["BookingID", "Customer ID", "BookingDate", "TableNo"])
bookings.head(5)

,BookingID,Customer ID,BookingDate,TableNo
0,1,720557985,2023-10-19,8
1,2,653530657,2023-10-09,4
2,3,908766799,2023-10-02,6
3,4,738734827,2023-10-04,2
4,5,809275246,2023-10-10,3


In [214]:
# Convert the Booking Date to a string in the format YYYY-MM-DD
bookings["BookingDate"] = bookings["BookingDate"].dt.strftime('%Y-%m-%d')

In [215]:
# Insert data into the bookings table
for i, row in bookings.iterrows():
    sql = "insert into bookings (BookingID, CustomerID, BookingDate, TableNo) values (%s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))

In [ ]:
# Update persistent changes to the database
connection.commit()

In [ ]:
# Close the cursor and connection
cursor.close()
connection.close()

The database is ready for experiment and analysis.